In [1]:
!pip install matplotlib
!pip install opencv-python
!pip install numpy


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: C:\Users\jonat\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: C:\Users\jonat\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: C:\Users\jonat\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [1]:
import matplotlib
matplotlib.use("TkAgg")
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os

In [ ]:
# Definimos función para mostrar imágenes
def imshow(img, new_fig=True, title=None, color_img=False, blocking=False, colorbar=False, ticks=False):
    if new_fig:  
        plt.figure()  # Crea una nueva figura si new_fig es True.
    if color_img:  
        plt.imshow(img)  # Muestra la imagen en colores.
    else:
        plt.imshow(img, cmap='gray')  # Muestra la imagen en escala de grises.
    plt.title(title)  # Agrega un título si se proporciona.
    if not ticks:  
        plt.xticks([]), plt.yticks([])  # Oculta los ejes si ticks es False.
    if colorbar:  
        plt.colorbar()  # Agrega una barra de color si colorbar es True.
    if new_fig:  
        plt.show(block=blocking)  # Muestra la figura y bloquea la ejecución según el parámetro blocking.

# Reconstrucción morfológica
def imreconstruct(marker, mask, kernel=None):
    # Asegurarse de que marker y mask sean del mismo tamaño y tipo
    if marker.shape != mask.shape:
        raise ValueError("El tamaño de 'marker' y 'mask' debe ser igual")  # Valida dimensiones iguales.
    if marker.dtype != mask.dtype:
        marker = marker.astype(mask.dtype)  # Convierte el tipo de datos de marker para que coincida con mask.

    # Definir el kernel si no se proporciona
    if kernel is None:
        kernel = np.ones((3, 3), np.uint8)  # Define un kernel de 3x3 si no se pasa uno.

    while True:
        expanded = cv2.dilate(marker, kernel)  # Realiza una dilatación sobre el marcador.
        expanded_intersection = cv2.bitwise_and(expanded, mask)  # Calcula la intersección con la máscara.
        if np.array_equal(marker, expanded_intersection):  # Verifica si no hay cambios en la iteración.
            break
        marker = expanded_intersection  # Actualiza el marcador con la intersección.
    return expanded_intersection  # Devuelve el resultado final de la reconstrucción.

# Relleno de huecos en imágenes binarias
def imfillhole(img):
    mask = np.zeros_like(img)  # Genera una máscara vacía con las mismas dimensiones que la imagen.
    mask = cv2.copyMakeBorder(mask[1:-1, 1:-1], 1, 1, 1, 1, cv2.BORDER_CONSTANT, value=int(255))  
    # Crea bordes constantes blancos alrededor de la máscara.
    marker = cv2.bitwise_not(img, mask=mask)  # Calcula el complemento de los bordes como marcador.
    img_c = cv2.bitwise_not(img)  # Obtiene el complemento de la imagen original como máscara.
    img_r = imreconstruct(marker=marker, mask=img_c)  # Realiza la reconstrucción usando el marcador y la máscara.
    img_fh = cv2.bitwise_not(img_r)  # Toma el complemento de la reconstrucción para obtener la imagen final.
    return img_fh  # Devuelve la imagen binaria con los huecos rellenos.

# Comparación de movimiento entre cuadros consecutivos
def se_mueve_no_se_mueve(foto, centroides):
    anterior = ''  # Inicializa la variable para almacenar los centroides del cuadro anterior.
    actual = ''  # Inicializa la variable para almacenar los centroides del cuadro actual.
    lista = []  # Lista para almacenar los identificadores de los objetos que no se han movido.
    for i in centroides:  # Recorre los datos de los centroides.
        if i[0] == (foto-1):  
            anterior = i  # Encuentra los centroides correspondientes al cuadro anterior.
        if i[0] == foto:
            actual = i  # Encuentra los centroides del cuadro actual.
            break
    if anterior == '' or actual == '':  # Si no se encontraron datos para uno de los cuadros, devuelve una lista vacía.
        return lista
    
    ant_cen = anterior[1]  # Obtiene los centroides del cuadro anterior.
    act_cen = actual[1]  # Obtiene los centroides del cuadro actual.
    for j in range(len(ant_cen)):  # Recorre todos los centroides del cuadro anterior.
        if (act_cen[j][1][0]-20.0) < ant_cen[j][1][0] < (act_cen[j][1][0]+20.0) and \
           (act_cen[j][1][1]-20.0) < ant_cen[j][1][1] < (act_cen[j][1][1]+20.0):  
           # Verifica si la posición del centroide actual está dentro de un rango cercano al anterior.
            lista.append(act_cen[j][0])  # Agrega el identificador del objeto a la lista si cumple la condición.
    return lista  # Devuelve la lista con los identificadores de los objetos estacionarios.


In [ ]:
def procesar_imagenes(num_foto, frame, num_foto_centroides_dados=[]):
    # FUNCIONALIDAD PRINCIPAL: Procesar una imagen para detectar dados y determinar el valor de sus caras.

    # Listado para almacenar el valor de las caras de los dados en cada cuadro.
    num_cara_caja = []

    # Convertir la imagen de BGR a RGB y luego a escala de grises.
    img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2GRAY)

    # Aplicar umbral para segmentar áreas de interés.
    _, thresh = cv2.threshold(img_gray, 100, 255, cv2.THRESH_BINARY)

    # Aplicar el detector de bordes Canny para identificar bordes generales.
    canny = cv2.Canny(img_gray, 50, 75)
    engrosar_bordes_generales = cv2.dilate(canny, np.ones((17, 17), np.uint8))

    # Rellenar huecos en los bordes detectados.
    relleno_dados = imfillhole(engrosar_bordes_generales)

    # Erosionar los bordes para eliminar ruido y reducir detalles.
    erocion_dado = cv2.erode(relleno_dados, np.ones((3, 3), np.uint8))

    # SEGMENTACIÓN PARA DETECTAR CÍRCULOS (CARAS DE LOS DADOS).
    canny_circulos = cv2.Canny(img_gray, 175, 255)
    engrosar_circulos = cv2.dilate(canny_circulos, np.ones((3, 3), np.uint8))
    relleno_circulos = imfillhole(engrosar_circulos)
    erocion_circulos = cv2.erode(relleno_circulos, np.ones((5, 5), np.uint8))

    # OBTENER BOUNDING BOXES DE LOS DADOS.
    num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(erocion_dado)

    # Filtrar componentes conectados basados en criterios geométricos y de área.
    indice_stats = []  # Almacena índices de las estadísticas que cumplen las condiciones.
    centroides_dados = []  # Almacena los centroides de los dados detectados.
    for i in range(1, num_labels):  # Omitir el fondo (label 0).
        x, y, w, h, area = stats[i]  # Coordenadas y área del bounding box.
        if 50 < w < 120 and 50 < h < 140:  # Filtrar por tamaño del bounding box.
            factor_forma = w / h
            if 0.5 < factor_forma < 2:  # Filtrar por proporción del bounding box.
                if 4500 < area < 11000:  # Filtrar por área.
                    indice_stats.append(i)
                    centroides_dados.append([i, centroids[i]])

    # Guardar información de los centroides si hay suficientes dados detectados.
    if len(centroides_dados) >= 5:
        num_foto_centroides_dados.append([num_foto, centroides_dados])

    # DETERMINAR SI LOS DADOS ESTÁN ESTACIONARIOS ENTRE CUADROS.
    if len(num_foto_centroides_dados) >= 2:
        lista = se_mueve_no_se_mueve(num_foto, num_foto_centroides_dados)

        # CALCULAR EL VALOR DE LA CARA PARA LOS DADOS ESTACIONARIOS.
        num_cara_caja = []
        for j in lista:
            x2, y2, w2, h2, area2 = stats[j]
            img_circulos = np.zeros_like(img_gray)  # Imagen vacía para análisis de círculos.
            img_circulos[y2:y2 + h2, x2:x2 + w2] = erocion_circulos[y2:y2 + h2, x2:x2 + w2]

            # Detectar componentes conectados en la región del dado.
            num_labels_2, labels_2, stats_2, centroids_2 = cv2.connectedComponentsWithStats(img_circulos)
            cara_dado = 0  # Inicializar contador para las caras del dado.
            for i_label in range(1, num_labels_2):
                area_circulos = stats_2[i_label, cv2.CC_STAT_AREA]
                if 30 < area_circulos < 150:  # Filtrar por tamaño de los círculos.
                    mask = (labels_2 == i_label).astype(np.uint8) * 255
                    contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
                    for contour in contours:
                        perimetro_circulo = cv2.arcLength(contour, True)
                        factor_forma_circulo = area_circulos / (perimetro_circulo ** 2)
                        if 0.050 < factor_forma_circulo < 0.10:  # Filtrar por forma circular.
                            cara_dado += 1
            num_cara_caja.append([j, cara_dado])

    # DIBUJAR BOUNDING BOXES Y ETIQUETAS SOBRE LOS DADOS.
    if len(indice_stats) >= 5:
        for k in indice_stats:
            x3, y3, w3, h3, area3 = stats[k]
            punto1 = (x3, y3)  # Esquina superior izquierda del bounding box.
            punto2 = (x3 + w3, y3 + h3)  # Esquina inferior derecha.
            if not num_cara_caja:
                # Dibujar bounding box en rojo si no se ha detectado cara.
                cv2.rectangle(img_rgb, punto1, punto2, color=(0, 0, 255), thickness=5)
            else:
                for g in num_cara_caja:
                    if k != g[0]:
                        cv2.rectangle(img_rgb, punto1, punto2, color=(0, 0, 255), thickness=5)
                    else:
                        cv2.rectangle(img_rgb, punto1, punto2, color=(0, 0, 255), thickness=5)
                        cv2.putText(img_rgb, f'Valor es: {g[1]}', (x3 - 10, y3 - 10), cv2.FONT_HERSHEY_SIMPLEX,
                                    fontScale=1, color=(0, 255, 0), thickness=2)

    # Convertir la imagen procesada de nuevo a BGR para guardar o visualizar.
    terminada = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2BGR)
    return terminada, num_foto_centroides_dados


In [ ]:
# os.makedirs("frames", exist_ok = True)  # Si no existe, crea la carpeta 'frames' en el directorio actual (comentado actualmente).

# --- Leer un video ------------------------------------------------
for num_video in range(1, 5):  # Itera sobre los nombres de los videos ('tirada_1.mp4' a 'tirada_4.mp4').
    cap = cv2.VideoCapture(f'tirada_{num_video}.mp4')  # Abre el archivo de video correspondiente.
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))  # Obtiene el ancho del video en píxeles.
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))  # Obtiene la altura del video en píxeles.
    fps = int(cap.get(cv2.CAP_PROP_FPS))  # Obtiene los cuadros por segundo (FPS) del video.
    n_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))  # Obtiene el número total de frames en el video.
    frame_number = 0  # Inicializa el contador de frames.
    
    # Crea un objeto para guardar el video procesado con las mismas propiedades del original.
    out = cv2.VideoWriter(f'Video{num_video}-procesado.mp4', cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))
    num_foto_centroides_dados = []  # Lista para almacenar información de los centroides de los dados por frame.
    bandera = True  # Bandera para determinar el área útil solo una vez.

    while cap.isOpened():  # Repite mientras el archivo de video esté abierto.
        ret, frame = cap.read()  # Lee el siguiente frame del video. 'ret' indica éxito (True/False).
        if ret == True:  # Verifica si se leyó el frame correctamente.
            if bandera:  # Si es el primer frame, calcula el área útil donde están los dados.
                ancho = int(len(frame[0]) / 2)  # Obtiene el punto medio horizontal del frame.
                cont = 0  # Contador para encontrar el límite superior de la región útil.
                for j in range(len(frame)):  # Itera sobre las filas de píxeles en el frame.
                    if frame[j][ancho][2] < 50:  # Comprueba si el canal rojo es menor a 50 (indicio de borde útil).
                        cont += 1  # Incrementa el contador si cumple la condición.
                bandera = False  # Cambia la bandera para no repetir este cálculo en siguientes frames.

            largo = len(frame[0])  # Obtiene el ancho total del frame.
            
            # Procesa la región útil del frame para detectar y analizar los dados.
            foto_recortada, num_foto_centroides_dados = procesar_imagenes(
                frame_number, frame[0:cont, 0:largo], num_foto_centroides_dados
            )

            frame_number += 1  # Incrementa el contador de frames procesados.
            frame[0:cont, 0:largo] = foto_recortada  # Actualiza la región útil del frame con los resultados procesados.
            out.write(frame)  # Escribe el frame procesado en el archivo de salida.
        else:  
            break  # Detiene el bucle si no hay más frames.

    cap.release()  # Libera el archivo de video original.
    out.release()  # Libera el archivo de video procesado.
    cv2.destroyAllWindows()  # Cierra todas las ventanas abiertas por OpenCV.
